In [17]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import io
import json

In [ ]:
# TODO:
'''
Using pandas load the imdb reviews csv file
Analyse its shape and columns
Check for missing data and remove those rows
'''

import pandas as pd
imdb = pd.read_csv(r"C:\Users\ajinkya\Downloads\IMDB_Dataset.csv")
print("dataset :\n",imdb)
print("Shape of dataset : ",imdb.shape)
print("columns in dataset : ",imdb.columns)

missing_data = imdb.isnull().sum()
print("missing data : \n",missing_data)

imdb_cleaned = imdb.dropna()
print("Cleaned dataset :",imdb_cleaned)
print("Cleaned shape : ",imdb_cleaned.shape)

In [19]:
#TODO : Complete the function to preprocess the text data

def preprocessing(sentence):
    imdb = sentence
    # First make the sentence lowercase
    imdb = imdb.lower()
    
    
    # Remove all html tags from the sentence i.e replace anything between <> with space
    # Hint use Regular Expression i.e. re.sub()
    imdb = re.sub(r'<.*?>',' ',imdb)
    
    
    # Remove all special characters i.e. anything other than alphabets and numbers. Replace them with space
    imdb = re.sub(r'[^a-zA-Z0-9\s]', ' ' , imdb)
    
    # Remove all single characters i.e. a-z and A-Z and Replace them with space
    imdb = re.sub(r'\b[a-zA-Z]\b', ' ', imdb)
    
    # Remove all multiple spaces and replace them with single space
    imdb = re.sub(r'\s+', ' ', imdb)
    
    
    # Use the nltk library to remove all stopwords from the sentence
    # stopwords are the words like and, the, is, are etc.
    import nltk 
    from nltk.corpus import stopwords
    
    stop_words = set(stopwords.words('english'))
    words = imdb.split()
    filtered_words = [word for word in words if word not in stop_words]
    imdb = ' '.join(filtered_words)
    
    
    
    # return the sentence
    return imdb
     # remove this line after writing the code

In [20]:
# TODO :
# Call the preprocessing function for each review in the dataframe and
# save the results in a new list of preprocessed_reviews
# This list will be your input to the neural network
# We will call this list as X from now on
preprocessed_series = imdb['review'].apply(preprocessing) 
X = preprocessed_series.tolist()

#for review in X:
#    print(review)

In [ ]:
# TODO :
# Convert sentiment column in the dataframe to numbers
# Convert positive to 1 and negative to 0 and store it in numpy array
# We will call this numpy array as y from now on

imdb['sentiment'] = imdb['sentiment'].map({'positive': 1, 'negative': 0})
print(imdb['sentiment'])
#convert into numpy array
y = imdb['sentiment'].to_numpy()


#print(y)
    


In [22]:
#a TODO : Split the data into training and testing (80-20 ratio)
# The train set will be used to train our deep learning models 
# while test set will be used to evaluate how well our model performs 
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

## Preparing embedding layer
Let's now write the script for our embedding layer. Embedding layer converts our textual data into numeric form. It is then **used as the first layer for the deep learning model like LSTM**.  
To know more about word embedding you may refer to following video
https://www.youtube.com/watch?v=9S0-OC4LFNo  
#### Tokenize the data

In [23]:
# Initialize the tokenizer
word_tokenizer = Tokenizer()

# TODO: Fit the tokenizer on the training data (X_train)
word_tokenizer.fit_on_texts(X_train)

# TODO: Convert training data to sequences of integers
# Hint: Use texts_to_sequences method
X_train_sequences = word_tokenizer.texts_to_sequences(X_train)

# TODO: Convert test data to sequences of integers
# Hint: Use texts_to_sequences method
X_test_sequences = word_tokenizer.texts_to_sequences(X_test)

# End TODO
# Saving the tokenizer in a json file (Already done for you)
# This will be used later for prediction on data in next assignments
tokenizer_json = word_tokenizer.to_json()
with io.open('b3_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))
    
# Vocab_length is the number of unique words in our dataset
# Adding 1 to store dimensions for words for which no pretrained word embeddings exist
vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

92308

#### Padding

In [24]:
# Padding all reviews to be of same length 'maxlen' words
maxlen = 100
# You can try different dimensions like 50, 100, 200 and 300
# and see how the model performs in next week

# TODO: Pad the training data sequences
# Hint: Use pad_sequences with 'post' padding and maxlen=maxlen
X_train_padded = pad_sequences(X_train_sequences, padding = 'post', maxlen = maxlen)

# TODO: Pad the test data sequences
# Hint: Use pad_sequences with 'post' padding and maxlen=maxlen
X_test_padded = pad_sequences(X_test_sequences, padding='post', maxlen = maxlen)


#### Glove Embedding

In [25]:
# Initialize an empty dictionary for embeddings
embeddings_dictionary = dict()

# Open the GloVe file (a2_glove.6B.100d.txt) with utf-8 encoding
glove_file = open(r'C:\Users\ajinkya\OneDrive - Indian Institute of Technology Bombay\Desktop\Codes_all\SOC_codes\codes\glove_embeddings.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

# TODO : Create an embedding matrix where each row corresponds to the index of the
# unique word in the dataset and each column corresponds to the word vector
# in the GloVe embedding 
# So the matrix will have vocab_length rows and maxlen columns
embedding_dim = 100
embedding_matrix = np.zeros((vocab_length, embedding_dim))

for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector


### Assignment_04

In [26]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
# TODO 1: Create a model
model = Sequential()
model.add(Embedding(input_dim=vocab_length, output_dim=embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(LSTM(units=128, return_sequences=False))
model.add(Dense(units=1, activation='sigmoid'))

In [28]:
# TODO 2: Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
# Optional: Print the model summary
model.summary()

In [ ]:

# TODO 3: Train the model
# Assuming y_train and y_test are defined and contain the target values
history = model.fit(X_train_padded, Y_train, batch_size=32, epochs=10, validation_split=0.2)

# TODO 4: Evaluate the model with the test data
loss, accuracy = model.evaluate(X_test_padded, Y_test)

In [ ]:
# TODO 5: Print the accuracy of the model
print(f"Test Accuracy: {accuracy * 100:.2f}%")
# Optional: Plot training and validation accuracy and loss
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [32]:
# TODO 6: Save the model
model.save('lstm_model.h5')

### 4_b

In [36]:
### 4_b
import pandas as pd
from keras.models import load_model
from keras_preprocessing.text import tokenizer_from_json
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# TODO 7: Load the model that you saved in the previous cell using load_model() function.
model = load_model(r'C:\Users\ajinkya\OneDrive - Indian Institute of Technology Bombay\Desktop\Codes_all\SOC_codes\codes\lstm_model.h5')

# TODO 8: Load the IMDB Dataset using pandas
# Assuming the dataset is stored in 'imdb_reviews.csv'
imdb_df = pd.read_csv(r'C:\Users\ajinkya\OneDrive - Indian Institute of Technology Bombay\Desktop\Codes_all\SOC_codes\Data\IMDB_Dataset.csv')

# Display the first few rows of the dataframe
print(imdb_df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [37]:
# TODO 9: Preprocess the data as you did in the previous assignment.
# Let's assume the review text is in a column named 'review'
reviews = imdb_df['review'].values

# Load the tokenizer from the json file
with open('b3_tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

In [38]:
# TODO 10: Tokenize the data using the tokenizer_from_json() function
review_sequences = tokenizer.texts_to_sequences(reviews)

# TODO 11: Pad the data using the pad_sequences() function
review_padded = pad_sequences(review_sequences, padding='post', maxlen=100)

# TODO 12: Make predictions on the data using the model.predict() function
predictions = model.predict(review_padded)

# TODO 13: For each review, if the prediction is stored in variable, 'prediction',
# then np.round(prediction*10,1) will give you the predicted rating
predicted_ratings = np.round(predictions * 10, 1)

# Add the predicted ratings to the dataframe
imdb_df['predicted_rating'] = predicted_ratings

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step


In [39]:
# Save the results back to a csv file for comparison
imdb_df.to_csv('imdb_reviews_with_predictions.csv', index=False)

# Display the first few rows of the updated dataframe
print(imdb_df.head())


                                              review sentiment  \
0  One of the other reviewers has mentioned that ...  positive   
1  A wonderful little production. <br /><br />The...  positive   
2  I thought this was a wonderful way to spend ti...  positive   
3  Basically there's a family where a little boy ...  negative   
4  Petter Mattei's "Love in the Time of Money" is...  positive   

   predicted_rating  
0               6.2  
1               9.8  
2               9.9  
3               1.5  
4               9.7  
